<a href="https://colab.research.google.com/github/ahinagangopadhyay/Machine-Learning/blob/main/WordEmbeddings_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install gensim nltk datasets scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 11.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 req

In [2]:
!pip install fasttext


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313504 sha256=c022f44aa88619486345df96da195b02150119f62d18ffb593ace089cdc20a76
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


In [4]:
!pip install numpy==1.23.5


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 99.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
pymc 5.22.0 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
xarray 2025.3.1 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
blosc2 3.3.3 requires numpy>=1.26, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 wh

In [2]:
import re
import fasttext
import tempfile
from gensim.models import Word2Vec
from sklearn.datasets import fetch_20newsgroups
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np

# Load dataset (20 Newsgroups subset for lightness)
data = fetch_20newsgroups(subset='all', categories=['sci.space', 'rec.autos'])
texts = data.data
labels = data.target

# Simple regex-based tokenization
def preprocess(texts):
    token_pattern = re.compile(r'\b\w+\b')
    return [token_pattern.findall(text.lower()) for text in texts]

tokenized_texts = preprocess(texts)

# Train-test split
X_train_tokens, X_test_tokens, y_train, y_test = train_test_split(
    tokenized_texts, labels, test_size=0.2, random_state=42
)

# =======================
# Word2Vec (Gensim)
# =======================
w2v_model = Word2Vec(sentences=X_train_tokens, vector_size=100, window=5, min_count=2, workers=4)

def get_avg_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

X_train_w2v = [get_avg_vector(doc, w2v_model) for doc in X_train_tokens]
X_test_w2v = [get_avg_vector(doc, w2v_model) for doc in X_test_tokens]

clf_w2v = LogisticRegression(max_iter=1000)
clf_w2v.fit(X_train_w2v, y_train)
preds_w2v = clf_w2v.predict(X_test_w2v)
acc_w2v = accuracy_score(y_test, preds_w2v)

# =======================
# FastText (direct)
# =======================

# Save training data to temporary file in FastText format
def save_for_fasttext(texts, labels):
    with tempfile.NamedTemporaryFile(mode='w+', delete=False) as f:
        for text, label in zip(texts, labels):
            line = f"__label__{label} " + " ".join(text) + "\n"
            f.write(line)
        return f.name

train_file = save_for_fasttext(X_train_tokens, y_train)
test_file = save_for_fasttext(X_test_tokens, y_test)

# Train FastText
ft_model = fasttext.train_supervised(train_file, epoch=25, lr=1.0, wordNgrams=2, verbose=0)

# Predict
def predict_fasttext(model, texts):
    labels = []
    for tokens in texts:
        joined = " ".join(tokens)
        pred = model.predict(joined)[0][0]
        labels.append(int(pred.replace("__label__", "")))
    return labels

preds_ft = predict_fasttext(ft_model, X_test_tokens)
acc_ft = accuracy_score(y_test, preds_ft)

# =======================
# Final Results
# =======================
print(f"Word2Vec Accuracy: {acc_w2v:.4f}")
print(f"FastText Accuracy: {acc_ft:.4f}")

Word2Vec Accuracy: 0.8712
FastText Accuracy: 0.9899


AttributeError: columns